<a href="https://colab.research.google.com/github/bhadreshpsavani/UnderstandingNLP/blob/master/AnalysingPretrainedLongFormerModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Performance of Longformer on SQAUD2

In [34]:
!pip install -q transformers
!pip install -q datasets

In [35]:
from transformers import squad_convert_examples_to_features
from transformers.data.processors.squad import SquadV2Processor, SquadFeatures
from tqdm.notebook import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_metric, load_dataset
device =  'cuda' if torch.cuda.is_available() else 'cpu'

In [43]:
def get_result(predictions, references, dataset='squad_v2'):
  squad_metric = load_metric(dataset)
  score = squad_metric.compute(predictions=predictions, references=references)
  return score

def get_validation_data(dataset='squad_v2'):
  datasets = load_dataset(dataset)
  valid_dataset = datasets['validation']
  return valid_dataset

def get_infernece(valid_dataset, distilbert_model, tokenizer, device):
  predictions=[]
  references=[]
  distilbert_model.to(device)
  for example in tqdm(valid_dataset):
    inputs = tokenizer(example['question'], example['context'], return_tensors="pt")
    inputs.to(device)
    output = model(**inputs)
    start_index = torch.argmax(output['start_logits'])
    end_index = torch.argmax(output['end_logits'])
    ans_ids = inputs['input_ids'][0][start_index :end_index+1]
    answer = tokenizer.decode(ans_ids)
    answer = tokenizer.clean_up_tokenization(answer).strip()
    answer = '' if answer=='<s>' else answer
    no_answer_probability = 1 if len(example['answers']['answer_start'])!=0 else 0
    """
    predictions = [{'prediction_text': '1976', 'id': '56e10a3be3433e1400422b22', 'no_answer_probability': 0.}]
    references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]
    """
    pred = {'prediction_text':answer, 'id': example['id'], 'no_answer_probability' : no_answer_probability}
    ref = {'answers': example['answers'] , 'id': example['id']}
    predictions.append(pred)
    references.append(ref)
  return predictions, references

In [37]:
def print_results(model, tokenizer, device, dataset_name='squad_v2'):
  validation_dataset = get_validation_data(dataset_name)
  predictions, references = get_infernece(validation_dataset, model, tokenizer, device)
  score = get_result(predictions, references, dataset_name)
  print(score)

## LongFormer

In [23]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/longformer-base-4096-finetuned-squadv2", use_fast=False)
model = AutoModelForQuestionAnswering.from_pretrained("mrm8488/longformer-base-4096-finetuned-squadv2")

In [24]:
print_results(model, tokenizer, device)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/9cac55034b086140f0649ecb5c604d09d7da2f2f5b73a90caa2e2bcc1f5cac09)



{'exact': 79.4660153288975, 'f1': 82.91266052065696, 'total': 11873, 'HasAns_exact': 77.64844804318489, 'HasAns_f1': 84.55162253066118, 'HasAns_total': 5928, 'NoAns_exact': 81.27838519764508, 'NoAns_f1': 81.27838519764508, 'NoAns_total': 5945, 'best_exact': 79.4660153288975, 'best_exact_thresh': 1.0, 'best_f1': 82.91266052065693, 'best_f1_thresh': 1.0}


In [40]:
del model
del tokenizer

## Longformer Results:
```
{'HasAns_exact': 77.64844804318489,
 'HasAns_f1': 84.55162253066118,
 'HasAns_total': 5928,
 'NoAns_exact': 81.27838519764508,
 'NoAns_f1': 81.27838519764508,
 'NoAns_total': 5945,
 'best_exact': 79.4660153288975,
 'best_exact_thresh': 1.0,
 'best_f1': 82.91266052065693,
 'best_f1_thresh': 1.0,
 'exact': 79.4660153288975,
 'f1': 82.91266052065696,
 'total': 11873}
```

## Distilbert

In [45]:
distilbert_path='twmkn9/distilbert-base-uncased-squad2'
distilbert_tokenizer = AutoTokenizer.from_pretrained(distilbert_path)
distilbert_model = AutoModelForQuestionAnswering.from_pretrained(distilbert_path)
distilbert_model.to(device)

RuntimeError: ignored

In [44]:
print_results(distilbert_model, distilbert_tokenizer, device)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/9cac55034b086140f0649ecb5c604d09d7da2f2f5b73a90caa2e2bcc1f5cac09)


RuntimeError: ignored

## SQUAD2 Results: 

| Model Name | F1 | EM |
| ---- | ---- | ---- | 
| Longformer | 82.91 | 79.466 | 